In [ ]:
import os
import glob
import datetime
from sklearn.model_selection import train_test_split
import torch
import segmentation_models_pytorch as smp
import numpy as np
import rasterio
from rasterio.windows import from_bounds
from rasterio.enums import Resampling as RasterioResampling
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader


# Hàm tải và tiền xử lý cặp ảnh DSM và DEM
def load_and_preprocess_pair(dsm_path, dem_path, terrain_threshold):
    try:
        with rasterio.open(dsm_path) as dsm_src, rasterio.open(dem_path) as dem_src:
            if dsm_src.crs != dem_src.crs: return None, None
            left = max(dsm_src.bounds.left, dem_src.bounds.left)
            bottom = max(dsm_src.bounds.bottom, dem_src.bounds.bottom)
            right = min(dsm_src.bounds.right, dem_src.bounds.right)
            top = min(dsm_src.bounds.top, dem_src.bounds.top)
            if left >= right or bottom >= top: return None, None
            window = from_bounds(left, bottom, right, top, dem_src.transform)
            dem_nodata = dem_src.nodata if dem_src.nodata is not None else -9999
            dsm_nodata = dsm_src.nodata if dsm_src.nodata is not None else -9999
            dem_data = dem_src.read(1, window=window, boundless=True, fill_value=dem_nodata).astype(np.float32)
            dsm_data = dsm_src.read(1, window=window, out_shape=dem_data.shape, boundless=True, fill_value=dsm_nodata, resampling=RasterioResampling.nearest).astype(np.float32)
            valid_mask = (dsm_data != dsm_nodata) & (dem_data != dem_nodata)
            label_mask = np.zeros_like(dem_data, dtype=np.float32)
            is_terrain = np.abs(dsm_data - dem_data) <= terrain_threshold
            label_mask[valid_mask & is_terrain] = 1.0
            if np.any(valid_mask):
                mean_val = np.mean(dsm_data[valid_mask])
                std_val = np.std(dsm_data[valid_mask])
                if std_val > 1e-6:
                    dsm_normalized = (dsm_data - mean_val) / std_val
                    dsm_normalized[~valid_mask] = 0
                else: dsm_normalized = np.zeros_like(dsm_data)
            else: dsm_normalized = np.zeros_like(dsm_data)
            return dsm_normalized, label_mask
    except Exception: return None, None

# Tạo dataset từ các file GeoTIFF lớn rồi chia thành các patch nhỏ
class GeoTiffPatchDataset(Dataset):
    def __init__(self, file_pairs, patch_size, terrain_threshold):
        self.file_pairs = file_pairs
        self.patch_size = patch_size
        self.terrain_threshold = terrain_threshold
        self.patches = self._create_patches()
    def _create_patches(self):
        patches = []
        print("Đang tạo các patch từ file ảnh lớn...")
        for dsm_path, dem_path in tqdm(self.file_pairs):
            dsm_full, mask_full = load_and_preprocess_pair(dsm_path, dem_path, self.terrain_threshold)
            if dsm_full is not None:
                img_height, img_width = dsm_full.shape
                for y in range(0, img_height, self.patch_size):
                    for x in range(0, img_width, self.patch_size):
                        dsm_patch = dsm_full[y:y+self.patch_size, x:x+self.patch_size]
                        mask_patch = mask_full[y:y+self.patch_size, x:x+self.patch_size]
                        pad_h = self.patch_size - dsm_patch.shape[0]; pad_w = self.patch_size - dsm_patch.shape[1]
                        if pad_h > 0 or pad_w > 0:
                            dsm_patch = np.pad(dsm_patch, ((0, pad_h), (0, pad_w)), mode='constant', constant_values=0)
                            mask_patch = np.pad(mask_patch, ((0, pad_h), (0, pad_w)), mode='constant', constant_values=0)
                        if np.any(mask_patch): patches.append((dsm_patch, mask_patch))
        return patches
    def __len__(self): return len(self.patches)
    def __getitem__(self, idx):
        dsm_patch, mask_patch = self.patches[idx]
        dsm_tensor = torch.from_numpy(dsm_patch).float().unsqueeze(0)
        mask_tensor = torch.from_numpy(mask_patch).float().unsqueeze(0)
        return dsm_tensor, mask_tensor

def train_one_epoch(model, dataloader, loss_fn_1, loss_fn_2, optimizer, device):
    model.train(); total_loss = 0; epoch_tp, epoch_fp, epoch_fn, epoch_tn = 0, 0, 0, 0
    progress_bar = tqdm(dataloader, desc="Training")
    for inputs, labels in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn_1(outputs, labels) + loss_fn_2(outputs, labels)
        loss.backward(); optimizer.step(); total_loss += loss.item()
        preds = (torch.sigmoid(outputs) > 0.5).long(); labels_long = labels.long()
        tp, fp, fn, tn = smp.metrics.get_stats(preds, labels_long, mode='binary')
        epoch_tp += tp.sum(); epoch_fp += fp.sum(); epoch_fn += fn.sum(); epoch_tn += tn.sum()
        progress_bar.set_postfix(loss=loss.item())
    avg_loss = total_loss / len(dataloader)
    epoch_iou = smp.metrics.iou_score(epoch_tp, epoch_fp, epoch_fn, epoch_tn, reduction='micro')
    return avg_loss, epoch_iou.item()

# Hàm đánh giá một epoch trên tập validation
def validate_one_epoch(model, dataloader, loss_fn_1, loss_fn_2, device):
    model.eval(); total_loss = 0; epoch_tp, epoch_fp, epoch_fn, epoch_tn = 0, 0, 0, 0
    with torch.no_grad():
        progress_bar = tqdm(dataloader, desc="Validating")
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = loss_fn_1(outputs, labels) + loss_fn_2(outputs, labels)
            total_loss += loss.item()
            preds = (torch.sigmoid(outputs) > 0.5).long(); labels_long = labels.long()
            tp, fp, fn, tn = smp.metrics.get_stats(preds, labels_long, mode='binary')
            epoch_tp += tp.sum(); epoch_fp += fp.sum(); epoch_fn += fn.sum(); epoch_tn += tn.sum()
            progress_bar.set_postfix(val_loss=loss.item())
    avg_loss = total_loss / len(dataloader)
    epoch_iou = smp.metrics.iou_score(epoch_tp, epoch_fp, epoch_fn, epoch_tn, reduction='micro')
    return avg_loss, epoch_iou.item()

# Lưu biểu đồ huấn luyện
def plot_and_save_history(history, save_path):

    # Tạo một figure lớn chứa 2 biểu đồ con
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))
    
    # Lấy số epochs từ độ dài của history
    epochs = range(1, len(history['train_loss']) + 1)

    # Biểu đồ 1: Loss
    ax1.plot(epochs, history['train_loss'], 'bo-', label='Training Loss')
    ax1.plot(epochs, history['val_loss'], 'ro-', label='Validation Loss')
    ax1.set_title('Training and Validation Loss')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True)

    # Biểu đồ 2: IoU
    ax2.plot(epochs, history['train_iou'], 'bo-', label='Training IoU')
    ax2.plot(epochs, history['val_iou'], 'ro-', label='Validation IoU')
    ax2.set_title('Training and Validation IoU')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('IoU')
    ax2.legend()
    ax2.grid(True)
    
    # Lưu toàn bộ figure
    plt.savefig(save_path)
    print(f"Biểu đồ quá trình huấn luyện đã được lưu tại: {save_path}")
    plt.close(fig)



#Cấu hình chung
BASE_DATA_PATH = "Project II"   #phải sửa lại path này cho phù hợp với hệ thống 
OUTPUT_DIR = "Project II/outputs" #phải sửa lại path này cho phù hợp với hệ thống 
LEARNING_RATE = 1e-4
BATCH_SIZE = 4 
NUM_EPOCHS = 100 # Số epoch cho MỖI thí nghiệm
PATCH_SIZE = 256
TERRAIN_THRESHOLD = 0.2
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

##Lên lịch train
experiments = [
    {'arch': 'Unet', 'backbone': 'timm-efficientnet-b7'},

]


def run_experiment(architecture, backbone, train_loader, val_loader):
    """
    Hàm này thực hiện một quy trình huấn luyện đầy đủ cho một model
    VÀ ghi lại, vẽ biểu đồ quá trình huấn luyện.
    """
    # Tạo tên thử nghiệm duy nhất dựa trên thời gian và kiến trúc
    timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    experiment_name = f"{timestamp}_{architecture}_{backbone}"
    print("\n" + "="*80)
    print(f"BẮT ĐẦU THÍ NGHIỆM: {experiment_name}")
    print("="*80 + "\n")

    # Khởi tạo model
    model_class = getattr(smp, architecture)
    model = model_class(
        encoder_name=backbone,
        encoder_weights="advprop",  # Thay sang imagenet nếu cần thiết
        in_channels=1,
        classes=1,
    ).to(DEVICE)
    
    # Khởi tạo các thành phần khác
    loss_fn_1 = smp.losses.DiceLoss(mode='binary')
    loss_fn_2 = smp.losses.SoftBCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    best_model_path = os.path.join(OUTPUT_DIR, f"{experiment_name}_best.pth")
    best_iou = 0.0

    # Lưu chỉ số qua các epoch
    history = {'train_loss': [], 'train_iou': [], 'val_loss': [], 'val_iou': []}

    # Vòng lặp huấn luyện
    for epoch in range(NUM_EPOCHS):
        print(f"\n--- Thí nghiệm: {experiment_name} | Epoch {epoch + 1}/{NUM_EPOCHS} ---")
        
        train_loss, train_iou = train_one_epoch(model, train_loader, loss_fn_1, loss_fn_2, optimizer, DEVICE)
        val_loss, val_iou = validate_one_epoch(model, val_loader, loss_fn_1, loss_fn_2, DEVICE)
        
        # Lưu lại các chỉ số của epoch vào history
        history['train_loss'].append(train_loss)
        history['train_iou'].append(train_iou)
        history['val_loss'].append(val_loss)
        history['val_iou'].append(val_iou)
        
        print(f"Epoch Summary: Train Loss: {train_loss:.4f} | Train IoU: {train_iou:.4f} | Val Loss: {val_loss:.4f} | Val IoU: {val_iou:.4f}")
        
        # Lưu model nếu IoU trên validation tốt hơn mô hình tốt nhất hiện tại
        if val_iou > best_iou:
            best_iou = val_iou
            torch.save(model.state_dict(), best_model_path)
            print(f"  -> Model mới tốt nhất được lưu tại: {best_model_path} (IoU: {best_iou:.4f})")

    # Vẽ và lưu biểu đồ sau khi huấn luyện xong
    graph_save_path = os.path.join(OUTPUT_DIR, f"{experiment_name}_history.png")
    plot_and_save_history(history, graph_save_path)
            
    print(f"\n--- KẾT THÚC THÍ NGHIỆM: {experiment_name} ---\n")

def main():
    """
    Hàm chính để điều phối toàn bộ quá trình.
    """
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    print(f"Sử dụng thiết bị: {DEVICE.upper()}")

    # Chuẩn bị dữ liệu
    DSM_DIR = os.path.join(BASE_DATA_PATH, 'DSM')
    DEM_DIR = os.path.join(BASE_DATA_PATH, 'DEM')
    dsm_pattern = os.path.join(DSM_DIR, '**', '*.TIF'); dem_pattern = os.path.join(DEM_DIR, '**', '*.TIF')
    dsm_files = sorted(glob.glob(dsm_pattern, recursive=True)); dem_files = sorted(glob.glob(dem_pattern, recursive=True))
    dem_dict = {os.path.basename(f).replace('dem', 'dsm'): f for f in dem_files}
    file_pairs = []; 
    for dsm_file in dsm_files:
        base_name = os.path.basename(dsm_file)
        if base_name in dem_dict: file_pairs.append((dsm_file, dem_dict[base_name]))
    train_val_pairs, _ = train_test_split(file_pairs, test_size=0.15, random_state=42)
    train_pairs, val_pairs = train_test_split(train_val_pairs, test_size=0.2, random_state=42)
    
    # Tạo Dataset
    train_dataset = GeoTiffPatchDataset(train_pairs, PATCH_SIZE, TERRAIN_THRESHOLD)
    val_dataset = GeoTiffPatchDataset(val_pairs, PATCH_SIZE, TERRAIN_THRESHOLD)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    # Vòng lặp chính duyệt qua các thí nghiệm 
    for experiment_params in experiments:
        run_experiment(
            architecture=experiment_params['arch'],
            backbone=experiment_params['backbone'],
            train_loader=train_loader,
            val_loader=val_loader
        )
    
    print("\n" + "="*80)
    print("ĐÃ HOÀN THÀNH TẤT CẢ CÁC THÍ NGHIỆM!")
    print("="*80)

# --- Chạy chương trình ---
if __name__ == "__main__":
    main()